<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#工具包导入&amp;数据读取" data-toc-modified-id="工具包导入&amp;数据读取-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>工具包导入&amp;数据读取</a></span><ul class="toc-item"><li><span><a href="#工具包导入" data-toc-modified-id="工具包导入-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>工具包导入</a></span></li><li><span><a href="#自定义损失函数" data-toc-modified-id="自定义损失函数-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>自定义损失函数</a></span></li><li><span><a href="#数据读取" data-toc-modified-id="数据读取-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>数据读取</a></span></li></ul></li><li><span><a href="#基于规则的思路" data-toc-modified-id="基于规则的思路-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>基于规则的思路</a></span><ul class="toc-item"><li><span><a href="#模型训练" data-toc-modified-id="模型训练-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#模型提交:" data-toc-modified-id="模型提交:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>模型提交:</a></span><ul class="toc-item"><li><span><a href="#乘上一个大于0的系数,39.69671(银牌,47/375)" data-toc-modified-id="乘上一个大于0的系数,39.69671(银牌,47/375)-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>乘上一个大于0的系数,39.69671(银牌,47/375)</a></span></li><li><span><a href="#直接提交:-39.33863(36/475)" data-toc-modified-id="直接提交:-39.33863(36/475)-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>直接提交: 39.33863(36/475)</a></span></li><li><span><a href="#乘上一个小于0的系数:--39.26182(32/475)" data-toc-modified-id="乘上一个小于0的系数:--39.26182(32/475)-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>乘上一个小于0的系数:  39.26182(32/475)</a></span></li></ul></li></ul></li><li><span><a href="#小结" data-toc-modified-id="小结-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>小结</a></span></li></ul></div>

# 工具包导入&数据读取
## 工具包导入

In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600

import os 
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import xgboost as xgb
import time
import datetime 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
 
from collections import Counter 
from statistics import mode 
    
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import json 
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from itertools import product
import ast 
tqdm.pandas()

/opt/conda/envs/zjpy36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

from keras.layers.normalization import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers 

import keras

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold

Using TensorFlow backend.


In [3]:
def init():
    np.random.seed = 0 
init()

## 自定义损失函数

In [20]:
def smape(x, y):
    if x == y:
        return 0
    else:
        return np.abs(x-y)/(x+y)
    
#median function ignoring nans
def safe_median(s):
    return np.median([x for x in s if ~np.isnan(x)])

## 数据读取

In [4]:
path   = '/data/Data_JieZhang/Web/'
key    = pd.read_csv(path + 'key_2.csv')
train  = pd.read_csv(path + 'train_2.csv') 

In [5]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-

In [6]:
YEAR_SHIFT     = 364      # number of days in a year, use multiple of 7 to be able to capture week behavior
PERIOD         = 49       # number of days for median comparison
PREDICT_PERIOD = 75       # number of days which will be predicted 

In [7]:
train[list(train.columns[-(YEAR_SHIFT + 2*PERIOD):])+['Page']].head()

,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19,2016-06-20,2016-06-21,2016-06-22,2016-06-23,2016-06-24,2016-06-25,2016-06-26,2016-06-27,2016-06-28,2016-06-29,2016-06-30,2016-07-01,2016-07-02,2016-07-03,2016-07-04,2016-07-05,2016-07-06,2016-07-07,2016-07-08,2016-07-09,2016-07-10,2016-07-11,2016-07-12,2016-07-13,2016-07-14,2016-07-15,2016-07-16,2016-07-17,2016-07-18,2016-07-19,2016-07-20,2016-07-21,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31,2016-08-01,2016-08-02,2016-08-03,2016-08-04,2016-08-05,2016-08-06,2016-08-07,2016-08-08,2016-08-09,2016-08-10,2016-08-11,2016-08-12,2016-08-13,2016-08-14,2016-08-15,2016-08-16,2016-08-17,2016-08-18,2016-08-19,2016-08-20,2016-08-21,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31,2016-09-01,2016-09-02,2016-09-03,2016-09-04,2016-09-05,2016-09-06,2016-09-07,2016-09-08,2016-09-09,2016-09-10,2016-09-11,2016-09-12,2016-09-13,2016-09-14,2016-09-15,2016-09-16,2016-09-17,2016-09-18,2016-09-19,2016-09-20,2016-09-21,2016-09-22,2016-09-23,2016-09-24,2016-09-25,2016-09-26,2016-09-27,2016-09-28,2016-09-29,2016-09-30,2016-10-01,2016-10-02,2016-10-03,2016-10-04,2016-10-05,2016-10-06,2016-10-07,2016-10-08,2016-10-09,2016-10-10,2016-10-11,2016-10-12,2016-10-13,2016-10-14,2016-10-15,2016-10-16,2016-10-17,2016-10-18,2016-10-19,2016-10-20,2016-10-21,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08,2016-11-09,2016-11-10,2016-11-11,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,2017-02-09,2017-02-10,2017-02-11,2017-02-12,2017-02-13,2017-02-14,2017-02-15,2017-02-16,2017-02-17,2017-02-18,2017-02-19,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28,2017-03-01,2017-03-02,2017-03-03,2017-03-04,2017-03-05,2017-03-06,2017-03-07,2017-03-08,2017-03-09,2017-03-10,2017-03-11,2017-03-12,2017-03-13,2017-03-14,2017-03-15,2017-03-16,2017-03-17,2017-03-18,2017-03-19,2017-03-20,2017-03-21,2017-03-22,2017-03-23,2017-03-24,2017-03-25,2017-03-26,2017-03-27,2017-03-28,2017-03-29,2017-03-30,2017-03-31,2017-04-01,2017-04-02,2017-04-03,2017-04-04,2017-04-05,2017-04-06,2017-04-07,2017-04-08,2017-04-09,2017-04-10,2017-04-11,2017-04-12,2017-04-13,2017-04-14,2017-04-15,2017-04-16,2017-04-17,2017-04-18,2017-04-19,2017-04-20,2017-04-21,2017-04-22,2017-04-23,2017-04-24,2017-04-25,2017-04-26,2017-04-27,2017-04-28,2017-04-29,2017-04-30,2017-05-01,2017-05-02,2017-05-03,2017-05-04,2017-05-05,2017-05-06,2017-05-07,2017-05-08,2017-05-09,2017-05-10,2017-05-11,2017-05-12,2017-05-13,2017-05-14,2017-05-15,2017-05-16,2017-05-17,2017-05-18,2017-05-19,2017-05-20,2017-05-21,2017-05-22,2017-05-23,2017-05-24,2017-05-25,2017-05-26,2017-05-27,2017-05-28,2017-05-29,2017-05-30,2017-05-31,2017-06-01,2017-06-02,2017-06-03,2017-0

In [8]:
train = pd.melt(train[list(train.columns[-(YEAR_SHIFT + 2*PERIOD):])+['Page']], id_vars='Page', var_name='date', value_name='Visits')

In [9]:
train.head()

,Page,date,Visits
0,2NE1_zh.wikipedia.org_all-access_spider,2016-06-06,14.0
1,2PM_zh.wikipedia.org_all-access_spider,2016-06-06,12.0
2,3C_zh.wikipedia.org_all-access_spider,2016-06-06,4.0
3,4minute_zh.wikipedia.org_all-access_spider,2016-06-06,9.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,2016-06-06,8.0


In [12]:
train['date']  = train['date'].astype('datetime64[ns]') 
LAST_TRAIN_DAY = train['date'].max()
train = train.groupby(['Page'])["Visits"].apply(lambda x: list(x))

In [13]:
train.head()

Page
!vote_en.wikipedia.org_all-access_all-agents                 [6.0, 3.0, 3.0, 4.0, 3.0, 2.0, 2.0, 6.0, nan, 9.0, 2.0, 1.0, 5.0, 5.0, 5.0, 4.0, 2.0, 6.0, 4.0, ...
!vote_en.wikipedia.org_all-access_spider                     [4.0, 2.0, 2.0, 0.0, 1.0, 0.0, 1.0, 3.0, nan, 6.0, 2.0, 1.0, 2.0, 2.0, 3.0, 0.0, 1.0, 4.0, 2.0, ...
!vote_en.wikipedia.org_desktop_all-agents                    [6.0, 3.0, 3.0, 4.0, 3.0, 2.0, 2.0, 5.0, nan, 6.0, 2.0, 1.0, 5.0, 5.0, 4.0, 4.0, 2.0, 6.0, 4.0, ...
"Awaken,_My_Love!"_en.wikipedia.org_all-access_all-agents    [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
"Awaken,_My_Love!"_en.wikipedia.org_all-access_spider        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
Name: Visits, dtype: object

# 基于规则的思路
## 模型训练

In [21]:
%%time
pred_dict = {}
count     = 0
scount    = 0

for page, row in tqdm(zip(train.index, train)):
    last_month       = np.array(row[-PERIOD:])          # 最新的49天
    slast_month      = np.array(row[-2*PERIOD:-PERIOD]) # 最新的49 - 98天
    prev_last_month  = np.array(row[PERIOD:2*PERIOD])   # 最早的49-98天
    prev_slast_month = np.array(row[:PERIOD])           # 最早的49天
    use_last_year    = False
    
    if ~np.isnan(row[0]):                                                                                                 # 如果该网页存在流量 
        #calculate yearly prediction error
        year_increase = np.median(slast_month)/np.median(prev_slast_month)                                                # 计算median趋势
        year_error    = np.sum(list(map(lambda x: smape(x[0], x[1]), zip(last_month, prev_last_month * year_increase)))) # 计算误差
        
        #calculate monthly prediction error
        smedian       = np.median(slast_month)   # 计算上一个月的中位数
        month_error   = np.sum(list(map(lambda x: smape(x, smedian), last_month)))  # 如果用上个月的误差进行填充,那么最终的结果
        
        #check if yearly prediction is better than median prediction in the previous period
        error_diff    = (month_error - year_error) / PERIOD
        if error_diff > 0.1: # 0.1可以自己调节
            scount        += 1
            use_last_year = True
    
    if use_last_year:
        last_year = np.array(row[2*PERIOD:2*PERIOD+PREDICT_PERIOD])
        preds     = last_year * year_increase              # consider yearly increase while using the last years visits
    else:
        preds   = [0] * PREDICT_PERIOD                       # 如果一年的不行，那么我们就用Fibonacci数列预测
        
        windows = np.array([2, 3, 4, 7, 11, 18, 29, 47]) * 7 # kind of fibonacci
        medians = np.zeros((len(windows), 7))
        for i in range(7):
            for k in range(len(windows)):
                # use 3-day window. for example, Friday: [Thursday, Friday, Saturday]
                array         = np.array(row[-windows[k]:]).reshape(-1, 7)
                s             = np.hstack([array[:, (i-1)%7], array[:, i], array[:, (i+1)%7]]).reshape(-1)
                medians[k, i] = safe_median(s)
        for i in range(PREDICT_PERIOD):
            preds[i] = safe_median(medians[:, i%7])
                
    pred_dict[page] = preds
    
    count += 1        
    if count % 1000 == 0:
        print(count, scount)

del train
print("Yearly prediction is done on the percentage:", scount/count)

1008it [00:16, 69.40it/s]

1000 48


2006it [00:32, 61.56it/s]

2000 124


3007it [00:47, 61.20it/s]

...

144000 4611


145007it [38:55, 65.52it/s]

145000 4624


145063it [38:56, 61.49it/s]

Yearly prediction is done on the percentage: 0.03188269924101942
CPU times: user 38min 57s, sys: 10.7 s, total: 39min 7s
Wall time: 38min 56s


In [22]:
test = key.copy() 
test['date'] = test.Page.apply(lambda a: a[-10:])
test['Page'] = test.Page.apply(lambda a: a[:-11])
test['date'] = test['date'].astype('datetime64[ns]')

In [23]:
test["date"] = test["date"].apply(lambda x: int((x - LAST_TRAIN_DAY).days) - 1) 

In [24]:
test.head()

,Page,Id,date
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,3
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,4
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,5
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,6


In [27]:
%%time
def func(row):
    return pred_dict[row["Page"]][row["date"]]

test["Visits"] = test.apply(func, axis=1) 

CPU times: user 3min 8s, sys: 660 ms, total: 3min 9s
Wall time: 3min 9s


In [28]:
test['Visits_cp'] = test['Visits'].values

## 模型提交: 
### 乘上一个大于0的系数,39.69671(银牌,47/375)

In [30]:
test['Visits'] = test['Visits'].values + test['Visits_cp'].values*0.03 # overestimating is usually better for smape
test.loc[test.Visits.isnull(), 'Visits'] = 0
test.Visits = test.Visits.round(4)
test[['Id','Visits']].to_csv('submission_rule.csv', index=False)

### 直接提交: 39.33863(36/475)

In [33]:
test['Visits'] = test['Visits_cp'].values #+ test['Visits_cp'].values*0.01 # overestimating is usually better for smape
test.loc[test.Visits.isnull(), 'Visits'] = 0
test.Visits = test.Visits.round(4)
test[['Id','Visits']].to_csv('submission_rule2.csv', index=False)

### 乘上一个小于0的系数:  39.26182(32/475)

In [34]:
test['Visits'] = test['Visits_cp'].values * 0.98#+ test['Visits_cp'].values*0.01 # overestimating is usually better for smape
test.loc[test.Visits.isnull(), 'Visits'] = 0
test.Visits = test.Visits.round(4)
test[['Id','Visits']].to_csv('submission_rule3.csv', index=False)

# 小结
本篇notebook我们深入观察了不同网页的信息，包括全局信息，全局流量的周期性；以及局部的网页信息还有这些一些代表性的网页，通过分析与观察，我们发现，

- 很多网页的流量存在明显的周期性（月周期，周周期和年周期）；
- 网页流量会出现突然上升的情况（spikes），也就是异常的情况等；
- 不同网页流量的波动比较大
- 有些网页流量存在明显的波动下降和波动上升的趋势；



